In [1]:
from DCNN import DCNN
import os
import librosa
import numpy as np
from torchinfo import summary
from torch import nn
from torch.optim import Adam

# Read Data

In [2]:
root = 'Data/genres_original'
genres = os.listdir(root)
x = []
y = []
length = []
sr = 22050
for genre in genres:
    genre_root = os.path.join(root, genre)
    audios = os.listdir(genre_root)
    for audio in audios:
        audio_path = os.path.join(genre_root, audio)
        signal, _ = librosa.load(audio_path, sr=sr)
        x.append(signal)
        length.append(len(signal))
        y.append(genres.index(genre))
min_length = min(length)

# Segment and Normalise 

In [3]:
for i in range(1000):
    x[i] = x[i][0:min_length]
    x[i] = librosa.util.normalize(x[i])
x = np.asarray(x)
y = np.asarray(y)
print(x.shape,y.shape)

(1000, 660000) (1000,)


In [4]:
seg_length = 19683
frame_num = int(x.shape[1]/seg_length)
preprocessed_x = x[:, :frame_num*seg_length].reshape(frame_num*x.shape[0],1,seg_length)
preprocessed_y = (y.reshape(y.shape[0],1)*np.ones((y.shape[0],frame_num))).reshape(y.shape[0]*frame_num)
print(preprocessed_x.shape,preprocessed_y.shape)

(33000, 1, 19683) (33000,)


# Model Construction

In [5]:
model = DCNN(10)
model.cuda()
loss_function = nn.CrossEntropyLoss()
opt = Adam(model.parameters(), lr=0.001)
summary(model,[(64,1,19683)])

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [ConLayer: 1, Conv1d: 2, BatchNorm1d: 2, ReLU: 2]